In [1]:
import pandas as pd

Create a Dataframe, with values as a list-of-lists and columns as a list

In [2]:
df = pd.DataFrame([[123,'xt23',20],[123,'q45',2],[123,'a89',25],[77,'q45',3],[77,'a89',30],[92,'xt23',24],[92,'m33',60],[92,'a89',28]], columns=['userid','product','price'])
df

,userid,product,price
0,123,xt23,20
1,123,q45,2
2,123,a89,25
3,77,q45,3
4,77,a89,30
5,92,xt23,24
6,92,m33,60
7,92,a89,28


If we want the maximum price anyone paid, we just do this:

In [3]:
df['price'].max()

60

In [6]:
df.price[df.price.idxmax()]

60

If we want the max price per user, we'll do a groupby. When we do that, it does the aggregation on each column seperately. So the value we get on the price column might not be for the product that we get on the product column - **problematic !**

In [8]:
df.groupby('userid').max()

,product,price
userid,,
77,q45,30
92,xt23,60
123,xt23,25


In [11]:
df.groupby('userid')[['price']].max() #why double quote?

,price
userid,
77,30
92,60
123,25


Just like max, we can do sum, etc. Pandas will smartly leave out columns for which that aggregation doesn't have meaning.

In [13]:
df.groupby('userid').sum()

,price
userid,
77,33
92,112
123,47


We can sort columns this way:

In [14]:
df.sort_values(by=['userid','product'])

,userid,product,price
4,77,a89,30
3,77,q45,3
7,92,a89,28
6,92,m33,60
5,92,xt23,24
2,123,a89,25
1,123,q45,2
0,123,xt23,20


We can sort and filter columns this way:

In [18]:
df.sort_values(by=['userid','product'])[['userid','price']]

,userid,price
4,77,30
3,77,3
7,92,28
6,92,60
5,92,24
2,123,25
1,123,2
0,123,20


Diff is another routine. It does a diff with value in the previous row, calculating the difference between the current row and previous row

In [19]:
df.sort_values(by=['userid','product'])[['userid','price']].groupby(['userid']).diff()

,price
4,NaN
3,-27.0
7,NaN
6,32.0
5,-36.0
2,NaN
1,-23.0
0,18.0


If we want the maximum price each user paid and the product associated with that price, we will sort, group and filter. Groupby will maintain the sort order within each group.
*(For SQL users: in SQL, you groupby and the sort, but in Pandas, it's easier to do it the other way around)*

** How to find the second largest? **

In [26]:
(df
 .sort_values(by=['userid','price'], ascending=[True, False])
 .groupby('userid')
 .head(1))

,userid,product,price
4,77,a89,30
6,92,m33,60
2,123,a89,25


In [27]:
df.sort_values(by=['userid','price'],ascending=False).groupby('userid').head(1)

,userid,product,price
2,123,a89,25
6,92,m33,60
4,77,a89,30


Adding a new column is easy:

In [28]:
df['website']=['Amazon','Amazon','NewEgg','NewEgg','NewEgg','Amazon','Amazon','Amazon']
df

,userid,product,price,website
0,123,xt23,20,Amazon
1,123,q45,2,Amazon
2,123,a89,25,NewEgg
3,77,q45,3,NewEgg
4,77,a89,30,NewEgg
5,92,xt23,24,Amazon
6,92,m33,60,Amazon
7,92,a89,28,Amazon


In [31]:
df.groupby(['userid','website']).sum().reset_index()

,userid,website,price
0,77,NewEgg,33
1,92,Amazon,112
2,123,Amazon,22
3,123,NewEgg,25


Below, we are going to do the same groupby as above. But if we set the as_index flag to "False" we get a flat table instead of the nested indexes

In [32]:
df3=df.groupby(['userid','website'],as_index=False).sum()
df3

,userid,website,price
0,77,NewEgg,33
1,92,Amazon,112
2,123,Amazon,22
3,123,NewEgg,25


Let's create a second table:

In [37]:
df2 = pd.DataFrame([[123,'USA'],[77,'Canada'],[93,'USA'], [100, 'Germany']], columns=['userid','country'])
df2

,userid,country
0,123,USA
1,77,Canada
2,93,USA
3,100,Germany


We can combine the two tables using a merge function. What it does is, it will do a pairwise comparision of every row in table1 with every row in table2 and if the "on" condition matches, it will create a single row with columns from both those matched rows.

Merge of two tables with 5 rows each can give as little as 0 rows and as much as 25 rows.

    [1,2,3,4,5] merged with [6,7,8,9,10] will give 0 rows
    [1,2,3,4,5] merged with [1,2,3,4,5] will give 5 rows
    [1,1,1,1,1] merged with [1,1,1,1,1] will give 25 rows

In [45]:
pd.merge(df,df2,on='userid', how='left')

,userid,product,price,website,country
0,123,xt23,20,Amazon,USA
1,123,q45,2,Amazon,USA
2,123,a89,25,NewEgg,USA
3,77,q45,3,NewEgg,Canada
4,77,a89,30,NewEgg,Canada
5,92,xt23,24,Amazon,NaN
6,92,m33,60,Amazon,NaN
7,92,a89,28,Amazon,NaN


**Different types of join** <br>
inner - intersection <br>
left - based on left table <br>
right - based on the right table

We can merge and then groupby to get what we want (Money spend on each website per country)

In [47]:
pd.merge(df,df2,on='userid').groupby(['country','website'])['price'].sum()

country  website
Canada   NewEgg     33
USA      Amazon     22
         NewEgg     25
Name: price, dtype: int64

We can also work with previously merged tables. Below we use df3 instead of df (scroll up to see what df3 is). The result is the same as the previous box.

In [48]:
pd.merge(df3,df2,on='userid').groupby(['country','website']).sum()

userid  price
country website               
Canada  NewEgg       77     33
USA     Amazon      123     22
        NewEgg      123     25

Let's add another column: purchase date

In [49]:
df['date']=['2015-01-12','2015-01-08','2015-01-06','2015-01-03','2015-01-05','2015-01-04','2015-01-07','2015-01-02']
df

,userid,product,price,website,date
0,123,xt23,20,Amazon,2015-01-12
1,123,q45,2,Amazon,2015-01-08
2,123,a89,25,NewEgg,2015-01-06
3,77,q45,3,NewEgg,2015-01-03
4,77,a89,30,NewEgg,2015-01-05
5,92,xt23,24,Amazon,2015-01-04
6,92,m33,60,Amazon,2015-01-07
7,92,a89,28,Amazon,2015-01-02


Here is a tricky task. For each row, I want the average purchase price for that user prior to that purchase. One option is to do some loops. But another solution is to just do a merge on itself and filter.

But first, let's review what a merge (or 'join' if you come from SQL) does. Say you merge two dataframes with 3 rows each, how many rows would you end up with? The answer is anywhere between 0 and 9.

Consider the following examples, where table x has users and the movies they like. And table y has users and the wines they line. And let's do a merge to come up with possible movie and wine pairings for any user. In case A, we get 0 rows, in case B, we get 3 rows and case C we get 9 rows.

In [50]:
dfx = pd.DataFrame([[1,'Godfather'],[2,'Amelie'],[3,'Chicago']],columns=['userid','movies'])
dfy = pd.DataFrame([[4,'red'],[5,'white'],[6,'pink']],columns=['userid','wines'])
dfm1=pd.merge(dfx,dfy,on='userid')
dfm1

,userid,movies,wines


In [51]:
dfx = pd.DataFrame([[1,'Godfather'],[2,'Amelie'],[3,'Chicago']],columns=['userid','movies'])
dfy = pd.DataFrame([[1,'red'],[2,'white'],[3,'pink']],columns=['userid','wines'])
dfm1=pd.merge(dfx,dfy,on='userid')
dfm1

,userid,movies,wines
0,1,Godfather,red
1,2,Amelie,white
2,3,Chicago,pink


In [54]:
dfx = pd.DataFrame([[1,'Godfather'],[1,'Amelie'],[1,'Chicago']],columns=['userid','movies'])
dfy = pd.DataFrame([[1,'red'],[1,'white'],[1,'pink']],columns=['userid','wines'])
dfm1=pd.merge(dfx,dfy,on='userid')
dfm1

#Produces all combinations of movie and wine

,userid,movies,wines
0,1,Godfather,red
1,1,Godfather,white
2,1,Godfather,pink
3,1,Amelie,red
4,1,Amelie,white
5,1,Amelie,pink
6,1,Chicago,red
7,1,Chicago,white
8,1,Chicago,pink


Now let's return to the original question: For each row, I want the average purchase price for that user prior to that purchase. Let's do a merge on itself and filter.

If we join a table on itself, for each row, you'll get every other purchase the user did.

In [55]:
df4=pd.merge(df[['userid','date']],df[['userid','price','date']],on='userid')
df4

,userid,date_x,price,date_y
0,123,2015-01-12,20,2015-01-12
1,123,2015-01-12,2,2015-01-08
2,123,2015-01-12,25,2015-01-06
3,123,2015-01-08,20,2015-01-12
4,123,2015-01-08,2,2015-01-08
5,123,2015-01-08,25,2015-01-06
6,123,2015-01-06,20,2015-01-12
7,123,2015-01-06,2,2015-01-08
8,123,2015-01-06,25,2015-01-06
9,77,2015-01-03,3,2015-01-03


Then we can filter out the purchases that not prior to the current purchase (notice the date sorting on users 123 and 92 are flipped; doesn't impact the work).

In [68]:
df4=df4[df4['date_x']>df4['date_y']]
df4

,userid,date_x,price,date_y
1,123,2015-01-12,2,2015-01-08
2,123,2015-01-12,25,2015-01-06
5,123,2015-01-08,25,2015-01-06
11,77,2015-01-05,3,2015-01-03
15,92,2015-01-04,28,2015-01-02
16,92,2015-01-07,24,2015-01-04
18,92,2015-01-07,28,2015-01-02


Then we can group by to get the average price that we wanted

In [74]:
df5 = df4.groupby(['userid','date_x'], as_index=False).mean()
df5

,userid,date_x,price
0,77,2015-01-05,3.0
1,92,2015-01-04,28.0
2,92,2015-01-07,26.0
3,123,2015-01-08,25.0
4,123,2015-01-12,13.5


In [75]:
df5.rename(columns={'price': 'avg_price_prior'}, inplace=True)
df5

,userid,date_x,avg_price_prior
0,77,2015-01-05,3.0
1,92,2015-01-04,28.0
2,92,2015-01-07,26.0
3,123,2015-01-08,25.0
4,123,2015-01-12,13.5


Finally, we merge with the original dataframe

In [80]:
df6 = df.merge(df5, left_on=['userid', 'date'], right_on=['userid', 'date_x'],  how='left')
df6

,userid,product,price,website,date,date_x,avg_price_prior
0,123,xt23,20,Amazon,2015-01-12,2015-01-12,13.5
1,123,q45,2,Amazon,2015-01-08,2015-01-08,25.0
2,123,a89,25,NewEgg,2015-01-06,NaN,NaN
3,77,q45,3,NewEgg,2015-01-03,NaN,NaN
4,77,a89,30,NewEgg,2015-01-05,2015-01-05,3.0
5,92,xt23,24,Amazon,2015-01-04,2015-01-04,28.0
6,92,m33,60,Amazon,2015-01-07,2015-01-07,26.0
7,92,a89,28,Amazon,2015-01-02,NaN,NaN
